## Example plotting following [ASP Docs Section 8.19](https://stereopipeline.readthedocs.io/en/latest/examples/aster.html#aster) for ASTER with and without mapprojection

Below are example `asp_plot` outputs following the processing in ASP Docs ASTER example [Section 8.19](https://stereopipeline.readthedocs.io/en/latest/examples/aster.html#aster).

We diverge from the documentation example data, instead retrieving the example data used in https://github.com/uw-cryo/asp_tutorials:

```bash
wget https://zenodo.org/record/7972223/files/AST_L1A_00307312017190728_20200218153629_19952.zip

mkdir dataDir

tar xvf AST_L1A_00307312017190728_20200218153629_19952.zip -C dataDir
```

---

Pre-process the images for stereo:

```bash
aster2asp dataDir -o out
```

to output:

```
out-Band3B.tif
out-Band3B.xml
out-Band3N.tif
out-Band3N.xml
```

---

Run stereo processing and point2dem without initial mapprojection:

```bash
parallel_stereo -t aster         \
  --stereo-algorithm asp_mgm     \
  --subpixel-mode 9              \
  --aster-use-csm                \
   out-Band3N.tif out-Band3B.tif \
   out-Band3N.xml out-Band3B.xml \
   out_stereo/run

point2dem -r earth --auto-proj-center \
  --errorimage \
  out_stereo/run-PC.tif
```

---

Followed by re-processing with the additional mapprojection step onto a low-res output:

```bash
point2dem -r earth --auto-proj-center \
  --tr 200 out_stereo/run-PC.tif      \
  -o out_stereo/run-200m

mapproject --tr 15 --aster-use-csm \
  out_stereo/run-200m-DEM.tif      \
  out-Band3N.tif out-Band3N.xml out-Band3N_proj.tif

mapproject --tr 15 --aster-use-csm \
  out_stereo/run-200m-DEM.tif      \
  out-Band3B.tif out-Band3B.xml out-Band3B_proj.tif

parallel_stereo -t aster                  \
  --stereo-algorithm asp_mgm              \
  --subpixel-mode 9                       \
  --aster-use-csm                         \
  out-Band3N_proj.tif out-Band3B_proj.tif \
  out-Band3N.xml out-Band3B.xml           \
  out_stereo_proj/run                     \
  out_stereo/run-200m-DEM.tif

point2dem -r earth --auto-proj-center \
  --errorimage \
  out_stereo_proj/run-PC.tif
```

---

The results with and without mapprojection are displayed below.

## Full report

Example command for full stereo-processing report

### No Mapprojection

In [1]:
directory = "/Users/ben/Dropbox/UW_Shean/asp_plot-test_data/docs_examples/docs_sect_8_19_aster/"
stereo_directory = "out_stereo/"

In [ ]:
!asp_plot \
  --directory $directory \
  --stereo_directory $stereo_directory \
  --dem_filename run-DEM.tif \
  --subset_km 5 \
  --plot_icesat True \
  --capture_date 2017-07-31 \
  --plot_geometry False


Processing ASP files in /Users/ben/Dropbox/UW_Shean/asp_plot-test_data/docs_examples/docs_sect_8_19_aster/


No reference DEM found in log files. Please supply the reference DEM you used during stereo processing (or another reference DEM) if you would like to see some difference maps.



ASP DEM: /Users/ben/Dropbox/UW_Shean/asp_plot-test_data/docs_examples/docs_sect_8_19_aster/out_stereo/run-DEM.tif


Using map projection from DEM: EPSG:32610


No reference DEM found in log files. Please supply the reference DEM you used during stereo processing (or another reference DEM) if you would like to see some difference maps.



ASP DEM: /Users/ben/Dropbox/UW_Shean/asp_plot-test_data/docs_examples/docs_sect_8_19_aster/out_stereo/run-DEM.tif

Figure saved to /Users/ben/Dropbox/UW_Shean/asp_plot-test_data/docs_examples/docs_sect_8_19_aster/tmp_asp_report_plots/00.png
Plotting DEM results. This can take a minute for large inputs.
Figure saved to /Users/ben/Dropbox/UW_Shean/asp_plot-test_data/doc

### Mapprojection

In [ ]:
stereo_directory = "out_stereo_proj/"

In [ ]:
!asp_plot \
  --directory $directory \
  --stereo_directory $stereo_directory \
  --dem_filename run-DEM.tif \
  --subset_km 5 \
  --plot_icesat True \
  --capture_date 2017-07-31 \
  --plot_geometry False

## Individual plots

Example modular usage of `asp_plot` for this example processing.

### Processing Parameters

In [ ]:
%load_ext autoreload
%autoreload 2

from asp_plot.processing_parameters import ProcessingParameters

In [ ]:
processing_parameters = ProcessingParameters(
    processing_directory=directory,
    stereo_directory=stereo_directory
)
processing_parameters_dict = processing_parameters.from_log_files()

print(f"Processed on: {processing_parameters_dict['processing_timestamp']}\n")

print(f"Reference DEM: {processing_parameters_dict['reference_dem']}\n")

print(f"Bundle adjustment ({processing_parameters_dict['bundle_adjust_run_time']}):\n")
print(processing_parameters_dict["bundle_adjust"])

print(f"\nStereo ({processing_parameters_dict['stereo_run_time']}):\n")
print(processing_parameters_dict["stereo"])

print(f"\nPoint2dem ({processing_parameters_dict['point2dem_run_time']}):\n")
print(processing_parameters_dict["point2dem"])

### Scene Plots

In [ ]:
from asp_plot.scenes import ScenePlotter

In [ ]:
plotter = ScenePlotter(
  directory,
  stereo_directory,
  title="Scenes"
)

plotter.plot_scenes()

### Stereo Plots

In [ ]:
from asp_plot.stereo import StereoPlotter

In [ ]:
plotter = StereoPlotter(
  directory, 
  stereo_directory
)

In [ ]:
plotter.title = "Hillshade"

plotter.plot_detailed_hillshade()

In [ ]:
plotter.title="Stereo Match Points"

plotter.plot_match_points()

In [ ]:
plotter.title = "Disparity (meters)"

plotter.plot_disparity(
  unit="meters",
  quiver=True,
)

In [ ]:
plotter.title = "Disparity (pixels)"

plotter.plot_disparity(
  unit="pixels",
  quiver=True,
)

In [ ]:
plotter.title = "Stereo DEM Results"

plotter.plot_dem_results()